# Motion segmentation

In [120]:
# For Kmeans algorithm. See PGM otherwise for handcrafted class
from sklearn.cluster import KMeans
# For loading matlab matrix file
from scipy.io import loadmat
# Hungarian algorithm
from scipy.optimize import linear_sum_assignment

#from scipy.sparse.linalg import eigs, eigsh
from scipy.stats import itemfreq
import numpy as np

In [121]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%precision %.5f
plt.rc('text', usetex=True)
plt.rc('font',**{'family':'serif','serif':['Palatino']})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [274]:
#Algorithm and error
from error_evaluation import *
from Ksubspace import *
from spectral_clustering import *
from SSC import *

Extrait de 

@article{elhamifar2013sparse,

  title={Sparse subspace clustering: Algorithm, theory, and applications},
  
  author={Elhamifar, Ehsan and Vidal, Rene},
  
  journal={IEEE transactions on pattern analysis and machine intelligence},
  
  volume={35},
  
  number={11},
  
  pages={2765--2781},
  
  year={2013},
  
  publisher={IEEE}
  
}


![article](real_data_article.png)

## Globals

In [123]:
DATA_DIR = "data/Hopkins155/"
NEEDLE = "_truth.mat"

## Data

In [124]:
import glob
import os

In [289]:
motion_data = {}
count_files = 0
# iglob for iterator
## Recursive=True to use **
for file in glob.iglob(DATA_DIR + "**/*" + NEEDLE, recursive = True):
    mat_name = file.split("/")[-1].split(NEEDLE)[0]
    try:
        # Load matlab matrix
        motion_data[mat_name] = loadmat(file)
        # Cast in float64 x
        motion_data[mat_name]['x'] = motion_data[mat_name]\
        ['x'].astype(np.float32, subok = True, copy = False)
        # Cast number of points in int
        motion_data[mat_name]['points'] = motion_data[mat_name]\
        ['points'][0, 0].astype(int)
        # Cast number of frames in int
        motion_data[mat_name]['frames'] = motion_data[mat_name]\
        ['frames'][0, 0].astype(int)
        # Substract 1 to matlab labels
        motion_data[mat_name]['s'] -= 1
    except OSError:
        print ('cannonot open ', file)
    else:
        count_files += 1
        motion_data[mat_name]["data"] = \
        np.transpose(motion_data[mat_name]['x'][:2, : :],
                     axes = (0, 2, 1)).reshape(2 * motion_data[mat_name]['frames'],
                                               motion_data[mat_name]["points"])
print ("Number of files loaded : {}".format(count_files))

Number of files loaded : 156


# Test zone

## Cars1

In [262]:
key = "cars1"

In [264]:
# Data
motion_data[key]["data"].shape

(40, 307)

In [271]:
# Labels
print ("Number of groups : {}".format(motion_data[key]["s"].max() + 1))
motion_data[key]["s"].shape

Number of groups : 2


(307, 1)

In [320]:
T = SSC(motion_data["1R2RC"]["data"], 2, 10e3, 10e2)

1.69614801026 1.99407192704 484.256500285
0.511734949762 0.643528089904 207.861132405
0.217496704768 0.282344936483 123.280939214
0.167552889282 0.210478847517 81.8168941357
0.13700920829 0.164225522038 62.5932389972
0.117276114132 0.135752015546 48.5273983596
0.103036918555 0.116401634107 40.1499659148
0.0923832391918 0.102637102252 34.0941217857
0.084374505234 0.0924114373625 29.3062367234
0.0781688787155 0.0845788036606 25.300193933
0.0729741218684 0.0783315199204 23.1130840066
0.0686457690233 0.0734291369028 20.6696775987
0.0649624838341 0.0690471668367 18.7660475172
0.0618796562703 0.0653307752079 16.9498955523
0.0590397391579 0.062197963205 16.0006301855
0.0565773830158 0.0593768994974 14.6492350547
0.0543778312098 0.056874766863 13.6647381573
0.0523819515019 0.0546521030512 12.7771369143
0.0505283262953 0.052592339441 12.1595551945
0.0487385636974 0.0507261941048 11.8654460315
0.0471392256453 0.0489880785819 11.1424646716
0.0457095308656 0.047349387385 10.4970551754
0.0442760980

KeyboardInterrupt: 